In [1]:
import heapq
from collections import defaultdict, deque
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [10]:

class ECommerceRecommendation:
    def __init__(self):
        self.user_product_graph = defaultdict(set)  # Graph: user -> {products}
        self.product_popularity = defaultdict(int)  # HashMap: product -> purchase count
        self.user_vector = defaultdict(lambda: np.zeros(100))  # User preference vectors

    def add_purchase(self, user, product):
        self.user_product_graph[user].add(product)
        self.product_popularity[product] += 1
        self.update_user_vector(user, product)

    def update_user_vector(self, user, product):
        """ Simulates updating user preference vectors (for ML similarity). """
        index = hash(product) % 100
        self.user_vector[user][index] += 1

    def recommend_products(self, user):
        """ Recommend products based on similar users (Graph BFS). """
        recommended = set()
        queue = deque([user])
        visited = set()
        
        while queue:
            current_user = queue.popleft()
            visited.add(current_user)
            
            for other_user in self.user_product_graph:
                if other_user != current_user and not other_user in visited:
                    shared_products = self.user_product_graph[current_user] & self.user_product_graph[other_user]
                    if shared_products:
                        recommended.update(self.user_product_graph[other_user] - shared_products)
                        queue.append(other_user)
        
        return list(recommended) if recommended else ["No recommendations"]

    def trending_products(self, top_n):
        """ Get top trending products using Max Heap. """
        return heapq.nlargest(top_n, self.product_popularity, key=self.product_popularity.get)

    def ml_recommend_products(self, user):
        """ Recommend products using ML-based user similarity (Cosine Similarity). """
        user_list = list(self.user_vector.keys())
        user_matrix = np.array([self.user_vector[u] for u in user_list])
        
        if user not in self.user_vector:
            return ["No recommendations"]
        
        similarities = cosine_similarity([self.user_vector[user]], user_matrix)[0]
        similar_users = [user_list[i] for i in np.argsort(similarities)[::-1] if user_list[i] != user]
        
        recommended = set()
        for sim_user in similar_users:
            recommended.update(self.user_product_graph[sim_user] - self.user_product_graph[user])
            if len(recommended) >= 5:
                break
        
        return list(recommended) if recommended else ["No recommendations"]


In [11]:
recommendation_system = ECommerceRecommendation()
recommendation_system.add_purchase("User1", "Laptop")
recommendation_system.add_purchase("User1", "Mouse")
recommendation_system.add_purchase("User2", "Laptop")
recommendation_system.add_purchase("User2", "Keyboard")
recommendation_system.add_purchase("User3", "Phone")
recommendation_system.add_purchase("User3", "Earphones")

print("Graph-based Recommendations for User1:", recommendation_system.recommend_products("User1"))
print("Top Trending Products:", recommendation_system.trending_products(2))
print("ML-based Recommendations for User1:", recommendation_system.ml_recommend_products("User1"))


Graph-based Recommendations for User1: ['Keyboard']
Top Trending Products: ['Laptop', 'Mouse']
ML-based Recommendations for User1: ['Keyboard', 'Earphones', 'Phone']
